# Irony Detection Benchmark on TweetEval

## Objective

This notebook benchmarks several approaches for irony detection on Twitter
using the TweetEval Irony dataset.

We evaluate models based on a trade-off between **performance** (F1-Score on the Irony class) and **inference efficiency** (VRAM usage, Latency).

**Approaches Compared:**
1.  **State-of-the-Art (Reference):** `cardiffnlp/twitter-roberta-base-2021-124m-irony` 
2.  **Efficient Fine-Tuning (Challenger):** `SetFit` with `sentence-transformers/paraphrase-mpnet-base-v2`
3.  **Classic ML + Embeddings:** `XGBoost` on `Jina-Embeddings-v3` (dim reduced)
4.  **LLM (Baseline):** `Qwen 2.5 7B` (Local inference) 

## Dataset: TweetEval (Irony)

We use the official **TweetEval** benchmark (Irony subset), derived from *SemEval-2018 Task 3*.  
It consists of tweets labelled as **0 (Non-Irony)** or **1 (Irony)**.

**Data Splits & Distribution:**

| Split | Total Samples | Non-Irony (0) | Irony (1) | Balance (Irony %) |
| :--- | :---: | :---: | :---: | :---: |
| **Train** | 2,862 | ~1,430 | ~1,432 | ~50% (Balanced) |
| **Validation** | 955 | ~470 | ~485 | ~50% (Balanced) |
| **Test** | **784** | **473** | **311** | **39.7% (Imbalanced)** |

> **⚠️ Critical Observation:** > The training set is artificially balanced, but the **Test set reflects a more realistic distribution** (Irony is the minority class ~40%).  
> Therefore, `Accuracy` is misleading. We prioritize **F1-Score on the Positive Class (Irony)**.

**Metrics:**
* **Primary:** `F1-Score (Irony)` - Measures the ability to correctly identify irony without false positives.
* **Secondary:** `Accuracy` - Global performance indicator.
* **Engineering:** `Inference Speed` & `VRAM footprint`.

---

In [1]:
import os
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb  

from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.utils import logging
from sklearn.metrics import accuracy_score, f1_score, classification_report

# --- 1. CONFIGURATION HARDWARE (RTX 4060 OPTIMIZATION) ---
# On fait taire les warnings non critiques de Transformers
logging.set_verbosity_error()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# La RTX 4060 supporte bfloat16 : c'est le standard actuel pour l'inférence efficace
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

# Seed pour la reproductibilité (Indispensable pour un benchmark)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

print(f"Hardware: {torch.cuda.get_device_name(0) if DEVICE == 'cuda' else 'CPU'}")
print(f"Precision: {DTYPE}")
print(f"Device: {DEVICE}")

# --- 2. DATA INGESTION ---
# Chargement du benchmark officiel "tweet_eval" subset "irony"
print("\n📥 Chargement du dataset TweetEval (Irony)...")
dataset = load_dataset("tweet_eval", "irony")

# Extraction des splits
train_ds = dataset["train"]
val_ds = dataset["validation"]
test_ds = dataset["test"]

print(f"Dataset chargé avec succès.")
print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")
print(f"Labels: {train_ds.features['label'].names} (0=Not_Irony, 1=Irony)")

c:\Users\anass\Desktop\irony-detection-nlp-benchmark\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware: NVIDIA GeForce RTX 4060 Laptop GPU
Precision: torch.bfloat16
Device: cuda

📥 Chargement du dataset TweetEval (Irony)...
Dataset chargé avec succès.
Train: 2862 | Val: 955 | Test: 784
Labels: ['non_irony', 'irony'] (0=Not_Irony, 1=Irony)


In [2]:
# --- 3. HELPER FUNCTIONS & UTILS ---
import time
import gc
import re

# Global leaderboard to accumulate results for the final report
leaderboard = []

def flush_memory():
    """
    Aggressively cleans up VRAM between models.
    Crucial for RTX 4060 (8GB) to avoid OOM (Out Of Memory) errors.
    """
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.ipc_collect()
    print("Memory flushed. VRAM ready for the next round.")

def evaluate_model(y_true, y_pred, model_name, inference_time=None):
    """
    Standardized evaluation function.
    Ensures fair comparison across all models using the same metrics.
    """
    acc = accuracy_score(y_true, y_pred)
    f1_irony = f1_score(y_true, y_pred, pos_label=1)
    
    print(f"\n📊 RESULTS: {model_name}")
    print(f"  Accuracy: {acc:.2%}")
    print(f"  F1-Score (Irony): {f1_irony:.2%}")
    if inference_time:
        print(f"  Inference Time: {inference_time:.2f}s")
        
    leaderboard.append({
        "Model": model_name,
        "Accuracy": acc,
        "F1_Irony": f1_irony,
        "Inference_Time_s": inference_time
    })
    
    print("-" * 40)
    print(classification_report(y_true, y_pred, target_names=["Not_Irony", "Irony"]))
    print("=" * 40)


def preprocess(text):
    """
    Cardiff models expect @user and http normalization.
    """
    new_text = [
    ]
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# --- APPLY TO ALL SPLITS ---
def preprocess_dataset(dataset):
    """Apply preprocessing to a HuggingFace dataset split."""
    return dataset.map(lambda x: {"text": preprocess(x["text"])})

print("🔄 Applying preprocessing...")

train_ds = preprocess_dataset(train_ds)
val_ds = preprocess_dataset(val_ds)
test_ds = preprocess_dataset(test_ds)

print("Helper functions loaded. Ready to benchmark.")

🔄 Applying preprocessing...


Map: 100%|██████████| 784/784 [00:00<00:00, 11953.80 examples/s]

Helper functions loaded. Ready to benchmark.


# Test Cardiff twitter-roberta-base-2021-124m-irony

In [ ]:
# =============================================================================
# EXPERIMENT 1: SOTA Reference — Cardiff RoBERTa (2021-124M)
# =============================================================================
# This is the state-of-the-art industrializable model for irony detection.
# Pre-trained on 124M tweets, then fine-tuned on TweetEval Irony.
# =============================================================================

MODEL_NAME_CARDIFF = "cardiffnlp/twitter-roberta-base-2021-124m-irony"

print("=" * 60)
print("EXPERIMENT 1: Cardiff RoBERTa (SOTA Reference)")
print("=" * 60)
print(f"Model: {MODEL_NAME_CARDIFF}")
print(f"Device: {DEVICE}")

# --- Load Model & Tokenizer ---
tokenizer_cardiff = AutoTokenizer.from_pretrained(MODEL_NAME_CARDIFF)
model_cardiff = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_CARDIFF)
model_cardiff = model_cardiff.to(DEVICE).eval()

# --- Identify Irony class ID (robust method) ---
irony_id = 1  # default
for k, v in model_cardiff.config.id2label.items():
    label = str(v).lower()
    if "irony" in label and "non" not in label and "not" not in label:
        irony_id = int(k)
        print(f"✅ Irony class detected: id={irony_id} → '{v}'")

# --- Prepare Test Data ---
test_texts = test_ds["text"]
test_labels = np.array(test_ds["label"], dtype=np.int64)

print(f"Test samples: {len(test_texts)}")

# --- Inference with timing ---
BATCH_SIZE = 64
probas = []

print("\nRunning inference...")
start_time = time.time()

with torch.inference_mode():
    for i in range(0, len(test_texts), BATCH_SIZE):
        batch = test_texts[i:i + BATCH_SIZE]
        
        # Tokenize
        encoded = tokenizer_cardiff(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(DEVICE)
        
        # Forward pass
        logits = model_cardiff(**encoded).logits
        probs = torch.softmax(logits, dim=-1)[:, irony_id]
        probas.append(probs.cpu().numpy())

inference_time = time.time() - start_time

# --- Predictions ---
probas = np.concatenate(probas)
y_pred_cardiff = (probas >= 0.5).astype(np.int64)

print(f"✅ Inference complete in {inference_time:.2f}s")
print(f"   Throughput: {len(test_texts) / inference_time:.1f} samples/sec")

# --- Evaluation (uses helper function) ---
evaluate_model(test_labels, y_pred_cardiff, "Cardiff RoBERTa-2021-124M (SOTA)")

# --- Cleanup ---
del model_cardiff, tokenizer_cardiff
flush_memory()

🏆 EXPERIMENT 1: Cardiff RoBERTa (SOTA Reference)
Model: cardiffnlp/twitter-roberta-base-2021-124m-irony
Device: cuda


ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

In [5]:
import sys
import subprocess

# On downgrade transformers vers une version stable "pré-panique" (fin 2024/2025)
print("🔧 Downgrading transformers to bypass CVE check...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==4.46.3"])

print("✅ Fini. REDÉMARRE LE KERNEL MAINTENANT (Bouton ↻ en haut).")

🔧 Downgrading transformers to bypass CVE check...
✅ Fini. REDÉMARRE LE KERNEL MAINTENANT (Bouton ↻ en haut).


In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score, classification_report

model_name = "cardiffnlp/twitter-roberta-base-2021-124m-irony"
device = "cuda" if torch.cuda.is_available() else "cpu"

ds = load_dataset("tweet_eval", "irony", split="test")
texts = ds["text"]
y_true = np.array(ds["label"], dtype=np.int64)  # 1=IRONY

tok = AutoTokenizer.from_pretrained(model_name)
mdl = AutoModelForSequenceClassification.from_pretrained(model_name).to(device).eval()

# trouve l'id de la classe "irony" via id2label (pas de mapping fragile)
irony_id = 1
for k, v in mdl.config.id2label.items():
    s = str(v).lower()
    if "irony" in s and "non" not in s and "not" not in s:
        irony_id = int(k)

proba = []
with torch.inference_mode():
    for i in range(0, len(texts), 64):
        batch = texts[i:i+64]
        enc = tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        p = torch.softmax(mdl(**enc).logits, dim=-1)[:, irony_id].cpu().numpy()
        proba.append(p)

proba = np.concatenate(proba)
y_pred = (proba >= 0.5).astype(np.int64)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1(IRONY):", f1_score(y_true, y_pred, pos_label=1))
print(classification_report(y_true, y_pred, target_names=["NOT_IRONY", "IRONY"], digits=4))


c:\Users\anass\Desktop\Tests\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accuracy: 0.7857142857142857
F1(IRONY): 0.7507418397626113
              precision    recall  f1-score   support

   NOT_IRONY     0.8622    0.7674    0.8121       473
       IRONY     0.6970    0.8135    0.7507       311

    accuracy                         0.7857       784
   macro avg     0.7796    0.7905    0.7814       784
weighted avg     0.7967    0.7857    0.7877       784



# Test Cardiff twitter-roberta-base-irony sur la dataset benshmark

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from sklearn.metrics import classification_report, accuracy_score
from tqdm.auto import tqdm
import torch

# 1. SETUP
device = 0 if torch.cuda.is_available() else -1
device_name = torch.cuda.get_device_name(0) if device == 0 else "CPU"
print(f"🔥 Hardware pour inférence : {device_name}")

# 2. CHARGEMENT DES DONNÉES (TEST SEULEMENT)
print("⏳ Chargement du Test Set TweetEval (Irony)...")
dataset = load_dataset("tweet_eval", "irony", split="test")
texts = dataset['text']
# 1 = Irony, 0 = Not Irony
y_true = dataset['label'] 

# 3. CHARGEMENT DU MODÈLE SOTA (CardiffNLP)
model_name = "cardiffnlp/twitter-roberta-base-irony"
print(f"🔧 Chargement du modèle SOTA : {model_name}")

# On utilise le pipeline pour aller vite et gérer le pré-traitement automatiquement
# Ce modèle renvoie généralement les labels "irony" et "non_irony"
pipe = pipeline("text-classification", model=model_name, device=device, tokenizer=model_name)

# 4. INFERENCE (PRÉDICTIONS)
print(f"🚀 Lancement de l'inférence sur {len(texts)} tweets...")
predictions = []

# On passe par batch pour optimiser le GPU
batch_size = 32
for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i + batch_size]
    results = pipe(batch)
    
    for res in results:
        # Le modèle Cardiff renvoie des labels textuels, il faut mapper vers 0/1
        # Mapping observé : 'irony' -> 1, 'non_irony' -> 0
        label_str = res['label']
        label_int = 1 if "irony" in label_str and "non" not in label_str else 0
        predictions.append(label_int)

# 5. RÉSULTATS
target_names = ["NOT_IRONY", "IRONY"]

print("\n" + "="*50)
print(f"📊 RÉSULTATS BASELINE ({model_name})")
print("="*50)
print(f"Accuracy: {accuracy_score(y_true, predictions):.2%}")
print("\n" + classification_report(y_true, predictions, target_names=target_names))

🔥 Hardware pour inférence : NVIDIA GeForce RTX 4060 Laptop GPU
⏳ Chargement du Test Set TweetEval (Irony)...
🔧 Chargement du modèle SOTA : cardiffnlp/twitter-roberta-base-irony
🚀 Lancement de l'inférence sur 784 tweets...


100%|██████████| 25/25 [00:23<00:00,  1.05it/s]



📊 RÉSULTATS BASELINE (cardiffnlp/twitter-roberta-base-irony)
Accuracy: 73.47%

              precision    recall  f1-score   support

   NOT_IRONY       0.75      0.85      0.79       473
       IRONY       0.71      0.56      0.63       311

    accuracy                           0.73       784
   macro avg       0.73      0.71      0.71       784
weighted avg       0.73      0.73      0.73       784



# Entrainer et tester SetFit (MPNet)

In [1]:
import gc
import torch
from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 1) Nettoyage
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Hardware:", torch.cuda.get_device_name(0) if device == "cuda" else "CPU")

# 2) Data
dataset = load_dataset("tweet_eval", "irony")
train_ds = dataset["train"]
val_ds   = dataset["validation"]
test_ds  = dataset["test"]

labels_names = ["NOT_IRONY", "IRONY"]

# 3) Modèle
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
model = SetFitModel.from_pretrained(
    model_id,
    device=device,
    labels=labels_names,
)

# 4) TrainingArguments (robuste)
# NOTE: SetFit default metric_for_best_model is "embedding_loss" and greater_is_better=False :contentReference[oaicite:5]{index=5}
args = TrainingArguments(
    output_dir="setfit_checkpoints_irony",
    batch_size=16,
    num_epochs=1,
    num_iterations=20,
    body_learning_rate=2e-5,

    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="embedding_loss",  # IMPORTANT: matches available metrics
    greater_is_better=False,

    use_amp=(device == "cuda"),  # AMP supported in SetFit embedding phase :contentReference[oaicite:6]{index=6}
    report_to="none",
    seed=42,
)

# 5) Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    metric="accuracy",  # used for SetFit evaluation (not for checkpoint selection in phase 1)
)

# 6) Train
trainer.train()

# 7) Test
preds = model.predict(test_ds["text"])
y_true = test_ds["label"]




c:\Users\anass\Desktop\Tests\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware: NVIDIA GeForce RTX 4060 Laptop GPU


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 114480
  Batch size = 16
  Num epochs = 1
  0%|          | 1/7155 [00:01<2:35:01,  1.30s/it]

{'embedding_loss': 0.5223, 'grad_norm': 3.421121120452881, 'learning_rate': 2.793296089385475e-08, 'epoch': 0.0}


  1%|          | 51/7155 [00:10<21:15,  5.57it/s] 

...

                                                   
100%|██████████| 7155/7155 [31:10<00:00,  4.86it/s]

{'eval_embedding_loss': 0.3744417726993561, 'eval_embedding_runtime': 146.8481, 'eval_embedding_samples_per_second': 260.133, 'eval_embedding_steps_per_second': 16.262, 'epoch': 1.0}


100%|██████████| 7155/7155 [31:15<00:00,  3.81it/s]


{'train_runtime': 1875.5603, 'train_samples_per_second': 61.038, 'train_steps_per_second': 3.815, 'train_loss': 0.047260442745151385, 'epoch': 1.0}

FINAL TEST RESULTS
Accuracy: 0.0000


ValueError: Mix of label input types (string and number)

In [3]:
import numpy as np
from sklearn.metrics import f1_score, classification_report, accuracy_score

labels_names = ["NOT_IRONY", "IRONY"]
label2id = {name: i for i, name in enumerate(labels_names)}

preds = model.predict(test_ds["text"])                 # strings
preds_int = np.array([label2id[p] for p in preds])     # ints
y_true = np.array(test_ds["label"], dtype=np.int64)    # ints

print("Accuracy:", accuracy_score(y_true, preds_int))
print("F1(IRONY=1):", f1_score(y_true, preds_int, pos_label=1, average="binary"))
print(classification_report(y_true, preds_int, target_names=labels_names, digits=4))


# 8) Save
save_path = "final_model_irony_mpnet_setfit"
model.save_pretrained(save_path)
print("Saved:", save_path)

Accuracy: 0.7678571428571429
F1(IRONY=1): 0.7422096317280453
              precision    recall  f1-score   support

   NOT_IRONY     0.8740    0.7188    0.7889       473
       IRONY     0.6633    0.8424    0.7422       311

    accuracy                         0.7679       784
   macro avg     0.7687    0.7806    0.7655       784
weighted avg     0.7904    0.7679    0.7704       784

Saved: final_model_irony_mpnet_setfit


In [8]:
from setfit import SetFitModel
import torch

path = "final_model_irony_mpnet_setfit"
model = SetFitModel.from_pretrained(path)

def count_torch_params(module) -> int:
    if module is None or not hasattr(module, "parameters"):
        return 0
    return sum(p.numel() for p in module.parameters())

# SetFit: body + head :contentReference[oaicite:6]{index=6}
body = getattr(model, "model_body", None)
head = getattr(model, "model_head", None)

n_body = count_torch_params(body)
n_head = count_torch_params(head)

print("Body type:", type(body))
print("Head type:", type(head))
print(f"Body params: {n_body:,}")
print(f"Head params: {n_head:,}")
print(f"Total torch params: {n_body + n_head:,}")

# Info utile: si head est sklearn, il aura plutôt get_params()
if head is not None and hasattr(head, "get_params"):
    print("Head seems sklearn-like. get_params keys:", list(head.get_params().keys())[:10])


Body type: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
Head type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
Body params: 109,486,464
Head params: 0
Total torch params: 109,486,464
Head seems sklearn-like. get_params keys: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'n_jobs', 'penalty', 'random_state']


In [6]:
from setfit import SetFitModel
import torch

# 1. Chargement
path = "final_model_irony_mpnet_setfit"
print(f"👑 Chargement du modèle depuis : {path}")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SetFitModel.from_pretrained(path).to(device)

# 2. Phrases de test
phrases = [
    "J'adore être coincé dans les bouchons le lundi matin, c'est ma passion.", 
    "Il fait vraiment beau aujourd'hui, quel plaisir.", 
    "Génial, mon ordinateur a planté juste avant la sauvegarde.", 
    "La réunion a duré 4 heures, c'était absolument fascinant de regarder le mur.",
    "Bravo champion, tu as encore oublié les clés."
]

# 3. Prédiction
preds = model.predict(phrases)
probas = model.predict_proba(phrases)

# 4. Affichage
print("\n" + "="*60)
print("🧠 LE DÉTECTEUR D'IRONIE EST PRÊT")
print("="*60)

for i, phrase in enumerate(phrases):
    pred_brute = preds[i] # Ici, c'est "NOT_IRONY" ou "IRONY"
    
    # LOGIQUE DE CORRECTION :
    # On détermine l'ID (0 ou 1) en fonction du texte reçu
    if str(pred_brute) == "NOT_IRONY":
        label_id = 0
        label_fr = "NON_IRONIQUE"
        emoji = "😐"
    else:
        label_id = 1
        label_fr = "IRONIQUE"
        emoji = "😏"
    
    # On récupère la confiance (score)
    # probas[i] est un tableau [score_non_irony, score_irony]
    if hasattr(probas[i][label_id], 'item'):
        score = probas[i][label_id].item()
    else:
        score = probas[i][label_id]
    
    print(f"📝 Phrase : \"{phrase}\"")
    print(f"🤖 Verdict : {emoji} {label_fr} (Confiance: {score:.1%})")
    print("-" * 30)

👑 Chargement du modèle depuis : final_model_irony_mpnet_setfit

🧠 LE DÉTECTEUR D'IRONIE EST PRÊT
📝 Phrase : "J'adore être coincé dans les bouchons le lundi matin, c'est ma passion."
🤖 Verdict : 😐 NON_IRONIQUE (Confiance: 99.9%)
------------------------------
📝 Phrase : "Il fait vraiment beau aujourd'hui, quel plaisir."
🤖 Verdict : 😐 NON_IRONIQUE (Confiance: 99.9%)
------------------------------
📝 Phrase : "Génial, mon ordinateur a planté juste avant la sauvegarde."
🤖 Verdict : 😐 NON_IRONIQUE (Confiance: 99.9%)
------------------------------
📝 Phrase : "La réunion a duré 4 heures, c'était absolument fascinant de regarder le mur."
🤖 Verdict : 😐 NON_IRONIQUE (Confiance: 99.9%)
------------------------------
📝 Phrase : "Bravo champion, tu as encore oublié les clés."
🤖 Verdict : 😐 NON_IRONIQUE (Confiance: 99.9%)
------------------------------


# XGBoost et Random Forest (Jina v3 - 512 dimensions)

In [1]:
import os
import sys
import gc
import numpy as np
import torch

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer
import joblib

# -----------------------------
# 0) Dependency check (must be BEFORE importing XGBClassifier)
# -----------------------------
try:
    import einops  # required by jina-embeddings-v3 remote code
except ImportError:
    print("Missing dependency: einops. Install with: pip install einops")
    sys.exit(1)

try:
    import xgboost
    from xgboost import XGBClassifier
    from xgboost.core import XGBoostError
except ImportError:
    print("Missing dependency: xgboost. Install with: pip install xgboost")
    sys.exit(1)

# -----------------------------
# 1) Repro + setup
# -----------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"Hardware: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
else:
    print("Hardware: CPU")

os.makedirs("models_backup", exist_ok=True)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# -----------------------------
# 2) Load data (TweetEval - irony)
# -----------------------------
print("Loading dataset: tweet_eval / irony ...")
dataset = load_dataset("tweet_eval", "irony")

X_train_text = dataset["train"]["text"]
y_train = np.array(dataset["train"]["label"], dtype=np.int64)

X_val_text = dataset["validation"]["text"]
y_val = np.array(dataset["validation"]["label"], dtype=np.int64)

X_test_text = dataset["test"]["text"]
y_test = np.array(dataset["test"]["label"], dtype=np.int64)

# -----------------------------
# 3) Encode with Jina v3 (classification LoRA + Matryoshka 512)
# -----------------------------
MODEL_ID = "jinaai/jina-embeddings-v3"
DIM = 768
TASK = "classification"
BATCH_SIZE = 32

def load_encoder(model_id: str, device: str, dim: int) -> SentenceTransformer:
    # Prefer truncate_dim if available; fallback if older SentenceTransformers
    try:
        return SentenceTransformer(
            model_id,
            trust_remote_code=True,
            device=device,
            truncate_dim=dim,
        )
    except TypeError:
        enc = SentenceTransformer(model_id, trust_remote_code=True, device=device)
        # Best-effort fallback
        try:
            enc.truncate_dim = dim
        except Exception:
            pass
        return enc

def encode_texts(enc: SentenceTransformer, texts, task: str, batch_size: int, dim: int) -> np.ndarray:
    # Prefer task adapter; fallback to no-task if something breaks
    try:
        emb = enc.encode(
            texts,
            convert_to_numpy=True,
            show_progress_bar=True,
            batch_size=batch_size,
            normalize_embeddings=True,
            task=task,
            truncate_dim=dim,  # also works as encode-time override in newer ST
        )
        return emb
    except Exception as e:
        print(f"[WARN] encode(task='{task}') failed ({type(e).__name__}: {e}). Falling back to default encode().")
        emb = enc.encode(
            texts,
            convert_to_numpy=True,
            show_progress_bar=True,
            batch_size=batch_size,
            normalize_embeddings=True,
            truncate_dim=dim,
        )
        return emb

print(f"Loading encoder: {MODEL_ID} | task={TASK} | dim={DIM} | device={device}")
encoder = load_encoder(MODEL_ID, device, DIM)

print("Encoding train...")
X_train = encode_texts(encoder, X_train_text, TASK, BATCH_SIZE, DIM)
print("Encoding val...")
X_val = encode_texts(encoder, X_val_text, TASK, BATCH_SIZE, DIM)
print("Encoding test...")
X_test = encode_texts(encoder, X_test_text, TASK, BATCH_SIZE, DIM)

# Free VRAM for tree models
del encoder
if device == "cuda":
    torch.cuda.empty_cache()
gc.collect()

# Merge train + val
X_full_train = np.vstack([X_train, X_val])
y_full_train = np.concatenate([y_train, y_val])

# -----------------------------
# 4) CV + scoring (TweetEval irony uses F1 for ironic class)
# -----------------------------
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
scorer = make_scorer(f1_score, pos_label=1, average="binary")

# -----------------------------
# 5) Random Forest (CPU)
# -----------------------------
print("\n[RF] RandomizedSearchCV...")
rf_params = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 10, 20, 30, 50],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["sqrt", "log2", None],
    "class_weight": ["balanced"],
}
rf = RandomForestClassifier(random_state=SEED, n_jobs=1)  # IMPORTANT: pas -1 ici :contentReference[oaicite:4]{index=4}

rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_params,
    n_iter=50,
    cv=cv,
    scoring=scorer,
    verbose=2,          # plus de logs :contentReference[oaicite:5]{index=5}
    random_state=SEED,
    n_jobs=-1,           # parallélise les fits CV
    pre_dispatch="1*n_jobs",
    error_score=np.nan,
    refit=True,
)
rf_search.fit(X_full_train, y_full_train)
best_rf = rf_search.best_estimator_

# -----------------------------
# 6) XGBoost (GPU if available; fallback to CPU if CUDA build missing)
# -----------------------------
print("\n[XGB] RandomizedSearchCV...")
xgb_params = {
    "n_estimators": [50, 100, 200, 300],
    "learning_rate": [0.005, 0.01, 0.03, 0.05, 0.1],
    "max_depth": [ 3, 5, 6, 8, None],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "gamma": [0.0, 0.1, 0.2, 0.5, 1.0],
    "reg_lambda": [0.0, 0.5, 1.0, 2.0],
    "reg_alpha": [0.0, 0.01, 0.05, 0.1, 0.5, 1.0],
    "max_bin": [128, 256],
}


def make_xgb(use_cuda: bool) -> XGBClassifier:
    return XGBClassifier(
        random_state=SEED,
        eval_metric="logloss",
        tree_method="hist",
        device="cuda" if use_cuda else "cpu",
        n_jobs=1,  # keep stable during CV
    )

def fit_xgb_search(use_cuda: bool):
    model = make_xgb(use_cuda)
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=xgb_params,
        n_iter=200,
        cv=cv,
        scoring=scorer,
        verbose=2,
        random_state=SEED,
        n_jobs=1,
    )
    search.fit(X_full_train, y_full_train)
    return search

use_cuda = (device == "cuda")
try:
    xgb_search = fit_xgb_search(use_cuda=use_cuda)
except XGBoostError as e:
    print(f"[WARN] XGBoost GPU failed ({e}). Falling back to CPU.")
    xgb_search = fit_xgb_search(use_cuda=False)

best_xgb = xgb_search.best_estimator_

# -----------------------------
# 7) Final evaluation + save
# -----------------------------
pred_rf = best_rf.predict(X_test)
pred_xgb = best_xgb.predict(X_test)

acc_rf = accuracy_score(y_test, pred_rf)
acc_xgb = accuracy_score(y_test, pred_xgb)

f1_irony_rf = f1_score(y_test, pred_rf, pos_label=1, average="binary")
f1_irony_xgb = f1_score(y_test, pred_xgb, pos_label=1, average="binary")

print("\n" + "=" * 70)
print(f"FINAL RESULTS — JINA V3 (task={TASK}, dim={DIM})")
print("=" * 70)

print("\n--- RANDOM FOREST ---")
print(f"Accuracy: {acc_rf:.4f} | F1(IRONY=1): {f1_irony_rf:.4f}")
print(classification_report(y_test, pred_rf, target_names=["NOT_IRONY", "IRONY"], digits=4))
joblib.dump(best_rf, f"models_backup/rf_jina_v3_{TASK}_{DIM}.joblib")

print("\n--- XGBOOST ---")
print(f"Accuracy: {acc_xgb:.4f} | F1(IRONY=1): {f1_irony_xgb:.4f}")
print(classification_report(y_test, pred_xgb, target_names=["NOT_IRONY", "IRONY"], digits=4))

print("\nDone.")


c:\Users\anass\Desktop\Tests\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware: NVIDIA GeForce RTX 4060 Laptop GPU
Loading dataset: tweet_eval / irony ...
Loading encoder: jinaai/jina-embeddings-v3 | task=classification | dim=768 | device=cuda


flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

Encoding train...


Batches: 100%|██████████| 90/90 [00:14<00:00,  6.23it/s]


...

Batches: 100%|██████████| 25/25 [00:03<00:00,  8.13it/s]



[RF] RandomizedSearchCV...
Fitting 3 folds for each of 50 candidates, totalling 150 fits

[XGB] RandomizedSearchCV...
Fitting 3 folds for each of 200 candidates, totalling 600 fits


c:\Users\anass\Desktop\Tests\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [23:44:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_bin=256, max_depth=6, n_estimators=200, reg_alpha=0.05, reg_lambda=0.5, subsample=0.9; total time=   5.3s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_bin=256, max_depth=6, n_estimators=200, reg_alpha=0.05, reg_lambda=0.5, subsample=0.9; total time=   5.1s
[CV] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.01, max_bin=256, max_depth=6, n_estimators=200, reg_alpha=0.05, reg_lambda=0.5, subsample=0.9; total time=   5.2s
[CV] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_bin=128, max_depth=6, n_estimators=100, reg_alpha=1.0, reg_lambda=0.0, subsample=1.0; total time=   4.1s
[CV] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_bin=128, max_depth=6, n_estimators=100, reg_alpha=1.0, reg_lambda=0.0, subsample=1.0; total time=   2.7s
[CV] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_bin=128, max_depth=6, n_estimators=100, reg_alpha=1.0, reg_lambda=0.0, subsa

TypeError: `_estimator_type` undefined.  Please use appropriate mixin to define estimator type.

In [5]:
import os
import sys
import gc
import numpy as np
import torch

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer
import joblib

# -----------------------------
# 0) Dependency check (must be BEFORE importing XGBClassifier)
# -----------------------------
try:
    import einops  # required by jina-embeddings-v3 remote code
except ImportError:
    print("Missing dependency: einops. Install with: pip install einops")
    sys.exit(1)

try:
    import xgboost
    from xgboost import XGBClassifier
    from xgboost.core import XGBoostError
except ImportError:
    print("Missing dependency: xgboost. Install with: pip install xgboost")
    sys.exit(1)

# -----------------------------
# 1) Repro + setup
# -----------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"Hardware: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
else:
    print("Hardware: CPU")

os.makedirs("models_backup", exist_ok=True)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# -----------------------------
# 2) Load data (TweetEval - irony)
# -----------------------------
print("Loading dataset: tweet_eval / irony ...")
dataset = load_dataset("tweet_eval", "irony")

X_train_text = dataset["train"]["text"]
y_train = np.array(dataset["train"]["label"], dtype=np.int64)

X_val_text = dataset["validation"]["text"]
y_val = np.array(dataset["validation"]["label"], dtype=np.int64)

X_test_text = dataset["test"]["text"]
y_test = np.array(dataset["test"]["label"], dtype=np.int64)

# -----------------------------
# 3) Encode with Jina v3 (classification LoRA + Matryoshka 512)
# -----------------------------
MODEL_ID = "jinaai/jina-embeddings-v3"
DIM = 256
TASK = "classification"
BATCH_SIZE = 32

def load_encoder(model_id: str, device: str, dim: int) -> SentenceTransformer:
    # Prefer truncate_dim if available; fallback if older SentenceTransformers
    try:
        return SentenceTransformer(
            model_id,
            trust_remote_code=True,
            device=device,
            truncate_dim=dim,
        )
    except TypeError:
        enc = SentenceTransformer(model_id, trust_remote_code=True, device=device)
        # Best-effort fallback
        try:
            enc.truncate_dim = dim
        except Exception:
            pass
        return enc

def encode_texts(enc: SentenceTransformer, texts, task: str, batch_size: int, dim: int) -> np.ndarray:
    # Prefer task adapter; fallback to no-task if something breaks
    try:
        emb = enc.encode(
            texts,
            convert_to_numpy=True,
            show_progress_bar=True,
            batch_size=batch_size,
            normalize_embeddings=True,
            task=task,
            truncate_dim=dim,  # also works as encode-time override in newer ST
        )
        return emb
    except Exception as e:
        print(f"[WARN] encode(task='{task}') failed ({type(e).__name__}: {e}). Falling back to default encode().")
        emb = enc.encode(
            texts,
            convert_to_numpy=True,
            show_progress_bar=True,
            batch_size=batch_size,
            normalize_embeddings=True,
            truncate_dim=dim,
        )
        return emb

print(f"Loading encoder: {MODEL_ID} | task={TASK} | dim={DIM} | device={device}")
encoder = load_encoder(MODEL_ID, device, DIM)

print("Encoding train...")
X_train = encode_texts(encoder, X_train_text, TASK, BATCH_SIZE, DIM)
print("Encoding val...")
X_val = encode_texts(encoder, X_val_text, TASK, BATCH_SIZE, DIM)
print("Encoding test...")
X_test = encode_texts(encoder, X_test_text, TASK, BATCH_SIZE, DIM)

# Free VRAM for tree models
del encoder
if device == "cuda":
    torch.cuda.empty_cache()
gc.collect()

# Merge train + val
X_full_train = np.vstack([X_train, X_val])
y_full_train = np.concatenate([y_train, y_val])

# -----------------------------
# 4) CV + scoring (TweetEval irony uses F1 for ironic class)
# -----------------------------
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)
scorer = make_scorer(f1_score, pos_label=1, average="binary")

# -----------------------------
# 5) Random Forest (CPU)
# -----------------------------
print("\n[RF] RandomizedSearchCV...")
rf_params = {
    "n_estimators": [100, 200, 250, 300],
    "max_depth": [None, 10, 30, 50, 70],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["sqrt", "log2", None],
    "class_weight": ["balanced"],
}
rf = RandomForestClassifier(random_state=SEED, n_jobs=1)  # IMPORTANT: pas -1 ici :contentReference[oaicite:4]{index=4}

rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_params,
    n_iter=70,
    cv=cv,
    scoring=scorer,
    verbose=10,          # plus de logs :contentReference[oaicite:5]{index=5}
    random_state=SEED,
    n_jobs=-1,           # parallélise les fits CV
    pre_dispatch="1*n_jobs",
    error_score=np.nan,
    refit=True,
)
rf_search.fit(X_full_train, y_full_train)
best_rf = rf_search.best_estimator_

# -----------------------------
# 6) XGBoost (GPU if available; fallback to CPU if CUDA build missing)
# -----------------------------
print("\n[XGB] RandomizedSearchCV...")
xgb_params = {
    "n_estimators": [100, 200, 250, 300, 350, 400, 450, 500],
    "learning_rate": [0.005, 0.01, 0.03, 0.05, 0.1],
    "max_depth": [ 3, 5, 6, 8, None],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "max_bin": [128, 256],
}


def make_xgb(use_cuda: bool) -> XGBClassifier:
    return XGBClassifier(
        random_state=SEED,
        eval_metric="logloss",
        tree_method="hist",
        device="cuda" if use_cuda else "cpu",
        n_jobs=1,  # keep stable during CV
    )

def fit_xgb_search(use_cuda: bool):
    model = make_xgb(use_cuda)
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=xgb_params,
        n_iter=200,
        cv=cv,
        scoring=scorer,
        verbose=10,
        random_state=SEED,
        n_jobs=1,
    )
    search.fit(X_full_train, y_full_train)
    return search

use_cuda = (device == "cuda")
try:
    xgb_search = fit_xgb_search(use_cuda=use_cuda)
except XGBoostError as e:
    print(f"[WARN] XGBoost GPU failed ({e}). Falling back to CPU.")
    xgb_search = fit_xgb_search(use_cuda=False)

best_xgb = xgb_search.best_estimator_

# -----------------------------
# 7) Final evaluation + save
# -----------------------------
pred_rf = best_rf.predict(X_test)
pred_xgb = best_xgb.predict(X_test)

acc_rf = accuracy_score(y_test, pred_rf)
acc_xgb = accuracy_score(y_test, pred_xgb)

f1_irony_rf = f1_score(y_test, pred_rf, pos_label=1, average="binary")
f1_irony_xgb = f1_score(y_test, pred_xgb, pos_label=1, average="binary")

print("\n" + "=" * 70)
print(f"FINAL RESULTS — JINA V3 (task={TASK}, dim={DIM})")
print("=" * 70)

print("\n--- RANDOM FOREST ---")
print(f"Accuracy: {acc_rf:.4f} | F1(IRONY=1): {f1_irony_rf:.4f}")
print(classification_report(y_test, pred_rf, target_names=["NOT_IRONY", "IRONY"], digits=4))
joblib.dump(best_rf, f"models_backup/rf_jina_v3_{TASK}_{DIM}.joblib")

print("\n--- XGBOOST ---")
print(f"Accuracy: {acc_xgb:.4f} | F1(IRONY=1): {f1_irony_xgb:.4f}")
print(classification_report(y_test, pred_xgb, target_names=["NOT_IRONY", "IRONY"], digits=4))

print("\nDone.")


Hardware: NVIDIA GeForce RTX 4060 Laptop GPU
Loading dataset: tweet_eval / irony ...
Loading encoder: jinaai/jina-embeddings-v3 | task=classification | dim=256 | device=cuda


flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

Encoding train...


Batches: 100%|██████████| 90/90 [00:16<00:00,  5.57it/s]


Encoding val...


Batches: 100%|██████████| 30/30 [00:04<00:00,  7.26it/s]


Encoding test...


Batches: 100%|██████████| 25/25 [00:04<00:00,  5.94it/s]



[RF] RandomizedSearchCV...
Fitting 3 folds for each of 70 candidates, totalling 210 fits

[XGB] RandomizedSearchCV...
Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV 1/3; 1/200] START colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7
[CV 1/3; 1/200] END colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7;, score=0.611 total time=   1.6s
[CV 2/3; 1/200] START colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7
[CV 2/3; 1/200] END colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7;, score=0.612 total time=   1.2s
[CV 3/3; 1/200] START colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7
[CV 3/3; 1/200] END colsample_bytree=0.8, learning_rate=0.01, max_bin=128, max_depth=6, n_estimators=250, subsample=0.7;, score=0.650 total time= 

In [ ]:
import json
import requests
import time
import os
from sklearn.metrics import classification_report, accuracy_score
from datasets import load_dataset

# --- CONFIGURATION ---
MODEL = "qwen2.5:7b-instruct"
URL = "http://localhost:11434/api/chat"
OUTPUT_FILE = "resultats_ironie_full.json"
BACKUP_EVERY = 50  # Sauvegarde automatique tous les 50 tweets

# --- 1. CHARGEMENT COMPLET (TRAIN + TEST) ---
print("⏳ Chargement de TOUT le dataset TweetEval (Irony)...")
dataset_train = load_dataset("tweet_eval", "irony", split="train")
dataset_test = load_dataset("tweet_eval", "irony", split="test")

# On fusionne tout dans une seule liste pour la nuit
full_data = []
print("📦 Fusion des données...")
for item in dataset_train:
    full_data.append({"text": item['text'], "label": item['label'], "split": "train"})
for item in dataset_test:
    full_data.append({"text": item['text'], "label": item['label'], "split": "test"})

print(f"✅ Prêt à traiter {len(full_data)} tweets cette nuit.")

# --- 2. FONCTION ROBUSTE ---
def classify_robust(text, max_retries=3):
    payload = {
        "model": MODEL,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are a strict irony detector. Output JSON only."},
            # Few-Shot Examples (Ta recette secrète)
            {"role": "user", "content": "Text: I love getting stuck in traffic.\nReturn JSON exactly: {\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}."},
            {"role": "assistant", "content": "{\"label\":\"IRONY\", \"confidence\": 0.99}"},
            {"role": "user", "content": "Text: The weather is beautiful today.\nReturn JSON exactly: {\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}."},
            {"role": "assistant", "content": "{\"label\":\"NOT_IRONY\", \"confidence\": 0.99}"},
            # Target
            {"role": "user", "content": f"Return JSON exactly: {{\"label\":\"IRONY\"|\"NOT_IRONY\",\"confidence\":0-1}}.\nText: {text}"}
        ],
        "options": {"temperature": 0}
    }
    
    for attempt in range(max_retries):
        try:
            r = requests.post(URL, json=payload, timeout=120)
            r.raise_for_status()
            content = r.json()["message"]["content"]
            
            # Parsing JSON artisanal mais solide
            start = content.find('{')
            end = content.rfind('}') + 1
            if start != -1 and end != -1:
                return json.loads(content[start:end]).get("label", "NOT_IRONY")
            
            # Fallback
            if "IRONY" in content and "NOT" not in content: return "IRONY"
            return "NOT_IRONY"
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"\n⚠️ Échec définitif sur ce tweet : {e}")
                return "ERROR"
            time.sleep(2) # On attend un peu avant de réessayer

# --- 3. EXÉCUTION & SAUVEGARDE CONTINUE ---
results = []
# Si le fichier existe déjà, on reprend (optionnel, ici on écrase pour faire simple)
if os.path.exists(OUTPUT_FILE):
    print(f"⚠️ Le fichier {OUTPUT_FILE} existe déjà, je vais ajouter à la suite...")

print("\n🚀 C'est parti ! Bonne nuit. (Ctrl+C pour arrêter si besoin)\n")

start_time = time.time()
y_true = []
y_pred = []

for i, item in enumerate(full_data):
    text = item['text']
    true_label = "IRONY" if item['label'] == 1 else "NOT_IRONY"
    
    # Classification
    pred_label = classify_robust(text)
    
    # On stocke tout
    result_entry = {
        "id": i,
        "text": text,
        "true_label": true_label,
        "pred_label": pred_label,
        "split": item['split']
    }
    results.append(result_entry)
    
    # Pour les métriques
    if pred_label != "ERROR":
        y_true.append(true_label)
        y_pred.append(pred_label)

    # Affichage minimaliste pour pas bourrer la console
    status = "✅" if pred_label == true_label else "❌"
    print(status, end="", flush=True)
    
    # Sauvegarde intermédiaire (Sécurité)
    if (i + 1) % BACKUP_EVERY == 0:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f" [Sauvegardé {i+1}/{len(full_data)}]", end="", flush=True)

# --- 4. RÉCAPITULATIF AU RÉVEIL ---
# Sauvegarde finale
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

duration = (time.time() - start_time) / 60
print(f"\n\n🏁 FINI ! Durée totale : {duration:.1f} minutes.")
print(f"📁 Résultats complets sauvegardés dans : {OUTPUT_FILE}")

print("\n" + "="*50)
print("📊 RAPPORT FINAL (Train + Test)")
print("="*50)
print(f"Accuracy Globale: {accuracy_score(y_true, y_pred):.2%}")
print(classification_report(y_true, y_pred, labels=["IRONY", "NOT_IRONY"]))

⏳ Chargement de TOUT le dataset TweetEval (Irony)...
📦 Fusion des données...
✅ Prêt à traiter 3646 tweets cette nuit.

🚀 C'est parti ! Bonne nuit. (Ctrl+C pour arrêter si besoin)

✅❌❌❌✅✅✅✅✅✅❌❌✅✅✅❌✅❌✅❌❌❌❌✅✅✅✅✅✅✅✅✅✅✅✅✅✅❌✅❌❌✅✅❌✅✅❌❌❌✅ [Sauvegardé 50/3646]✅✅❌❌❌❌✅✅❌✅❌✅✅❌✅❌❌❌❌❌✅✅✅✅❌✅✅✅✅✅✅❌✅✅✅✅❌❌✅❌❌✅✅✅❌✅✅❌✅✅ [Sauvegardé 100/3646]✅✅✅✅✅✅❌❌❌❌✅✅❌❌✅✅✅✅✅✅❌✅❌❌✅✅❌✅❌✅✅✅✅❌✅❌❌✅✅✅✅✅✅✅❌❌✅✅❌✅ [Sauvegardé 150/3646]❌❌❌✅❌✅✅✅✅✅✅❌✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅❌✅✅✅✅❌✅✅✅✅✅❌✅❌❌✅✅✅❌❌ [Sauvegardé 200/3646]✅❌❌❌✅❌✅✅✅✅✅❌❌✅❌✅✅❌✅❌✅✅❌✅❌❌❌✅❌✅✅❌❌✅❌✅❌✅✅✅✅✅✅✅✅❌❌❌❌❌ [Sauvegardé 250/3646]✅✅✅✅✅✅✅❌✅✅❌❌✅✅❌❌❌✅✅✅✅❌✅❌✅❌✅❌❌✅✅❌❌❌✅❌✅✅✅❌❌✅❌❌❌✅❌✅✅❌ [Sauvegardé 300/3646]❌✅✅❌❌✅❌❌❌✅❌✅✅❌✅❌❌❌✅✅✅❌❌❌✅❌✅✅✅✅✅✅❌❌❌✅✅❌❌✅✅❌❌❌✅❌✅❌✅✅ [Sauvegardé 350/3646]✅✅✅✅✅✅❌✅✅✅✅✅❌❌✅❌❌✅❌✅✅❌❌❌✅✅✅✅✅✅✅✅❌✅✅❌✅✅❌❌✅❌✅✅❌❌✅❌✅❌ [Sauvegardé 400/3646]✅✅❌✅❌✅✅❌❌❌✅✅❌✅✅❌✅✅✅❌❌✅❌❌✅✅❌✅❌❌✅✅✅✅✅✅✅✅❌❌❌✅✅✅✅❌✅✅✅✅ [Sauvegardé 450/3646]✅✅❌✅✅✅❌❌✅❌✅❌✅✅❌✅✅✅✅✅✅✅✅❌✅✅✅✅✅❌✅❌❌❌✅✅❌❌✅❌✅✅✅✅✅✅✅❌❌✅ [Sauvegardé 500/3646]❌✅✅✅❌✅✅❌✅❌❌✅✅✅✅❌❌❌✅✅✅❌✅✅❌❌✅✅✅❌❌❌✅✅❌❌✅❌✅✅✅✅❌✅✅❌❌❌❌✅ [Sauvegardé 550/3646]✅✅❌✅✅✅✅✅✅✅✅✅✅✅❌❌❌✅✅❌✅✅✅❌❌❌❌✅✅